# HW11 and 12: Scientific Claim Verification

**<span style="color:red">Important Instructions, read carefully!</span>** 

* Remember that these homework work as a completion grade. Deadline for this notebook is **June, 8th**. The homework is structured in two parts: (1) Information Retrieval from a fact base and (2) claim verification. In case you already submitted 9 homeworks and want to only submit a 10th notebook, it is fine to just do the first part. 

* In this notebook, we will build an automated claim verification system for scientific claims based on the [SciFact dataset](https://arxiv.org/abs/2004.14974). 

* In case you need additional computational resources (GPUs), please get in touch. It is possible to solve the homework (on a downsampled dataset) without these. If you want to build really cool systems, you probably want to use the whole dataset and train models which require compute not feasible on your local machine -- get in touch.

* Next, the best models to date on this dataset perform poorly. If you build a system yielding competitive scores, it is possible to do follow-up work! There exists a [global leaderboard](https://leaderboard.allenai.org/scifact/submissions/public) where you can submit test set results if you like. The [baseline system](https://arxiv.org/abs/2004.14974) achieves around 40% F1, we have built a system in January and obtain 55% F1, the [current state of the art](https://arxiv.org/pdf/2010.11930.pdf) is at around 65% F1 -- there exists ample room for improvements.

* You can find the github repo for SciFact with additional information (and possibly example code which could be usueful for solving this exercise) [here](https://github.com/allenai/scifact).

* For the first part of the assignment, we don't expect you to train your own transformer model, but you obviously can. For the second part of the assignment, we expect you to train your own textual entailment model.

* Lastly, We don't expect you to have a competitive system at the end of this homework, anything works as long as you have put in some effort.

**All instructions provided can be substituted by your own ideas and only serve as a rough guideline for how to tackle the task!**

* If you want, it is also possible to build a similar system for question answering (e.g. [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/)), automated fact checking (e.g. [Climate-FEVER](https://www.sustainablefinance.uzh.ch/en/research/climate-fever.html), [FEVER](https://fever.ai/)) or other involved NLP tasks completely freestyle on a dataset of your choice. If so, please get in touch with a suggestion.

* You are allowed to use any tools and help you can find online to tackle this task.

**An Example from SciFact**

Consider the claim *"Consumption of whole fruits increases the risk of type 2 diabetes."*

We are given a fact base of 5000 abstracts and have to find evidence from the fact base which either supports or refutes the claim. In this example, the goal is to find the following sentence from the corpus:

*'Greater consumption of specific whole fruits, particularly blueberries, grapes, and apples, is significantly associated with a lower risk of type 2 diabetes, whereas greater consumption of fruit juice is associated with a higher risk.'*

It is easy to see that this sentence contradicts the claim. The goal of the task is to return all sentences in the fact base which contradict or support a claim -- with the corresponding label. In this case, we would return 
"evidence": {"1974176": [{"sentences": [11], "label": "CONTRADICT"}
where "1974176" is the doc_id of the abstract we found the evidence in, and it is the 11th sentence in that abstract.

In [2]:
# obtain the data


# name="https://scifact.s3-us-west-2.amazonaws.com/release/latest/data.tar.gz"
# !wget $name
#
# !tar -xvf data.tar.gz
# !rm data.tar.gz

**Part 1 of the Assignment: Information Retrieval**

In this section, we will build a document retrieval system which takes as input a claim and returns a number of candidate abstracts which are similar to the claim. Commonly, we start with a recall-oriented system which returns abstracts likely to contain evidence sentences. Then, we follow up with a more advanced model which selects only relevant sentences from the retrieved abstracts.

In [3]:
# install some helper utils
# !pip install jsonlines

In [4]:
import itertools
import jsonlines

# load the corpus (the fact base with the abstracts)
corpus = {str(doc['doc_id']): doc for doc in itertools.islice(iter(jsonlines.open("data/corpus.jsonl")), 1000)}




# load the claims

# if you don't want to work with GPUs , you probably want to only consider 200/50 train/dev examples
# otherwise, this exercise might take too long

cpu_only = True
if cpu_only:
    claims_train = [claim for claim in jsonlines.open("data/claims_train.jsonl") if claim["evidence"]][:200]
    claims_dev = [claim for claim in jsonlines.open("data/claims_dev.jsonl") if claim["evidence"]][:50]
else:
    claims_train = [claim for claim in jsonlines.open("data/claims_train.jsonl") if claim["evidence"]][:200]
    claims_dev = [claim for claim in jsonlines.open("data/claims_dev.jsonl") if claim["evidence"]][:50]

print (len(claims_train))
print (len(claims_dev))

200
50


**Inspect the data**

Let's have a look at the corpus first. We see that every abstract has a unique doc_id, the title of the paper, the abstract (sentences are already tokenized) and a flag "structured" which is not relevant for this task.

In [5]:
print ("number of documents in the corpus", len(corpus))
# print (corpus["1974176"])
# print (corpus["1974176"].keys())
# dict_keys(['doc_id', 'title', 'abstract', 'structured'])
# abstract is a list of sentences

number of documents in the corpus 1000


Next, we look at an example claim in more detail. We find that it has a unique id, the claim as a string and annotated evidence. The evidence is a dictionairy where each key points to the abstract in the corpus. The values are a list where each entry contains the sentence number in the corresponding abstract and a label whether this sentence contradicts or supports the claim.

In [6]:
print(claims_train[0])
print(claims_train[0].keys())

{'id': 2, 'claim': '1 in 5 million in UK have abnormal PrP positivity.', 'evidence': {'13734012': [{'sentences': [4], 'label': 'CONTRADICT'}]}, 'cited_doc_ids': [13734012]}
dict_keys(['id', 'claim', 'evidence', 'cited_doc_ids'])


**Random Baseline for Abstract Retrieval**

As mentioned before, we need a system which retrieves abstracts from the corpus. Some ideas of how to tackle this include 
* create document embeddings via TF-IDF, SBERT, universal sentence encoder or any embedding technique you like. Embedd each claim and each abstract. Then find the closest abstracts for each claim
* use BM25 for document retrieval
* do something else which works

We provide a random baseline and evaluate recall for this method. Not surprisingly, this does not work well.

In [7]:
# random baseline
import random
def retrieve(claim, corpus, k):
    return random.sample(corpus.keys(), k=k)

retrieved_documents = []
for k in (3,5,10):
    for claim in claims_train:
        result = retrieve(claim["claim"], corpus, k)
        claim[f"doc_ids_k{k}"] = result

    for claim in claims_dev:
        result = retrieve(claim["claim"], corpus, k)
        claim[f"doc_ids_k{k}"] = result

In [8]:
# evaluate
def evaluate(claims, k, method = None):
    if method == 1:
        doc_ids_key = f"doc_ids_k{k}_1"
    elif method == 2:
        doc_ids_key = f"doc_ids_k{k}_2"
    else:
        doc_ids_key = f"doc_ids_k{k}"
    TP, FP, FN = 0, 0, 0
    for claim in claims:
        # relevant abstracts
        if claim["evidence"]:
            true_abstracts = set(claim["evidence"].keys())
            retrieved_abstracts = set(claim[doc_ids_key])
            TP += len(true_abstracts.intersection(retrieved_abstracts))
            FN += len(true_abstracts.difference(retrieved_abstracts))
            FP += len(retrieved_abstracts.difference(true_abstracts))
        else:
            FP += len(claim[doc_ids_key])
    try:
        pr = TP / (TP + FP)
        rc = TP / (TP + FN)
        f1 = 2 * pr * rc / (pr + rc)
    except ZeroDivisionError as e:
        print(e)
        pr, rc, f1 = 0,0,0
    print ("precision",pr, "recall",rc, "f1",f1)

for k in (3,5,10):
    print (f"train claims_{k}")
    evaluate(claims_train,k)
    print (f"dev claim_{k}")
    evaluate(claims_dev,k)


train claims_3
float division by zero
precision 0 recall 0 f1 0
dev claim_3
float division by zero
precision 0 recall 0 f1 0
train claims_5
precision 0.001 recall 0.004524886877828055 f1 0.0016380016380016379
dev claim_5
float division by zero
precision 0 recall 0 f1 0
train claims_10
float division by zero
precision 0 recall 0 f1 0
dev claim_10
float division by zero
precision 0 recall 0 f1 0


In [9]:
##TODO create your own abstract retrieval system

# Try with universal sentence encoder
I limited the corpus to 100 since calculating the embeddings was taking to long otherwise.

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()

import tensorflow_hub as hub
from tqdm import tqdm
import numpy as np

from numpy.linalg import norm

def cosine_similarity(a,b):
    return np.dot(a, b)/(norm(a)*norm(b))


module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
    return model(input)

# preprocess corpus and claims
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    for i in tqdm(corpus):
        sent_embs = embed(corpus[i]['abstract'])
        corpus[i]['abstract_sent_embedding'] = session.run(sent_embs)

        abstr_emb = embed(['. '.join(corpus[i]['abstract'])])
        corpus[i]['abstract_embedding'] = session.run(abstr_emb)

    # for idx, claim in tqdm(enumerate(claims_train)):
    #     emb = embed(claim['claim'])
    #     claims_train[idx]['claim_embed'] = session.run(emb)
    for idx, claim in tqdm(enumerate(claims_dev), total=len(claims_dev)):
        emb = embed([claim['claim']])
        claims_dev[idx]['claim_embed'] = session.run(emb)

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


100%|██████████| 50/50 [03:23<00:00,  4.07s/it]


In [ ]:
def retrieve(claim, corpus, k, sent_emb_method:bool = False):
    sims = {}
    for i in corpus:
        if sent_emb_method:
            abstr_sims = [
                cosine_similarity(
                    claim['claim_embed'].squeeze(), sent_emb.squeeze()
                )
                for sent_emb in corpus[i]['abstract_sent_embedding']
            ]

            sims[i] = max(abstr_sims)
        else:
            sims[i] = cosine_similarity(claim['claim_embed'].squeeze(), corpus[i]['abstract_embedding'].squeeze())

    sorted_sims= dict(sorted(sims.items(),key= lambda x:x[1],reverse=True))
    # print('claim',claim['claim'])
    # print('abstract', corpus[list(sorted_sims)[0]]['abstract'])
    return list(sorted_sims)[:k]


for k in (3,5,10):
    # for claim in claims_train:
    #     result = retrieve(claim["claim_embed"], corpus, k)
    #     claim[f"doc_ids_{k}"] = result

    for claim in claims_dev:
        claim[f"doc_ids_k{k}_1"] = retrieve(claim, corpus, k, sent_emb_method=True)
        claim[f"doc_ids_k{k}_2"] = retrieve(claim, corpus, k, sent_emb_method=False)


In [ ]:
##TODO evaluate your system. 
# If it operates on document level, we suggest to evaluate your system with k=3, k=5, k=10 retrieved documents

# else, evaluate it using some reasonable method

I am trying two methods to retrieve the most relevant abstracts to each claim:
- Method 1: Retrieve the abstract which contains the sentence which embedding has the highest cosine similarity with the embedding of the claim.
- Method 2: Retrieve the abstract which embedding has the highest cosine similarity with the embedding of the claim.

In [ ]:
for k in (3,5,10):
    # print (f"train claims_{k}")
    # evaluate(claims_train,k)
    print (f"dev claim_{k}")
    evaluate(claims_dev,k, method = 1)
    evaluate(claims_dev,k, method = 2)

In [ ]:
# save your results, we suggest for k=3, which makes the rest of this exercise less time consuming

k=3
# for claim in claims_train:
#     result = retrieve(claim["claim"], corpus, k)
#     claim["doc_ids"] = result


    
import json
# with open("data/claims_train_with_retrieved_documents.jsonl", "w") as outfile:
#     for claim in claims_train:
#         json.dump(claim, outfile)
#         outfile.write("\n")
        
# with open("data/claims_dev_with_retrieved_documents.jsonl", "w") as outfile:
#     for claim in claims_dev:
#         json.dump(claim, outfile)
#         outfile.write("\n")

**Sentence Retrieval**

Now, we have candidate documents for every claim. As we have seen before, the precision achieved is not very convincing. So, we train a second module which takes a claim and a sentence as input and decides whether this sentence is possible evidence which supports or verifies the claim. This is just another pairwise sentence classification task and is usually tackled as a binary classification.

* If you want to train your own model on CPU, we suggest to use [distilbert](https://huggingface.co/distilroberta-base) (which took me 20 minutes to fine-tune for one epoch on CPU). If you have access to GPUs, there's a variety of models to choose from, e.g. [here](https://huggingface.co/transformers/pretrained_models.html) or [here](https://huggingface.co/models).

* We also provide a model [here](https://www.dropbox.com/s/mh3lrg3z626d0xw/scibert_model.zip?dl=0) which is a BertForSequenceClassification checkpoint fine-tuned from [SciBERT](https://huggingface.co/allenai/scibert_scivocab_uncased) which you could use in this task. The model is trained to predict class 1 for annotated evidence sentences and class 0 for randomly sampled negative examples. You can download this model with 

* wget https://www.dropbox.com/s/mh3lrg3z626d0xw/scibert_model.zip?dl=0

* You can use any other model/method you like if you think it works reasonably well on this specific task

* We again provide a random baseline for demonstration purposes

In [ ]:
# continue with results from method 1
doc_ids_key = f"doc_ids_k{k}_1"

In [ ]:
# get pretrained model
!wget https://www.dropbox.com/s/mh3lrg3z626d0xw/scibert_model.zip?dl=0

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("scibert_finetuned_on_scifact")
model = AutoModelForSequenceClassification.from_pretrained("scibert_finetuned_on_scifact")
model.eval()

In [ ]:
# random baseline

import numpy as np
from tqdm import tqdm

for claim in tqdm(claims_dev):
    doc_ids = claim[doc_ids_key]
    predicted_evidence = {}
    for doc_id in doc_ids:
        sentences = corpus[doc_id]["abstract"]
        predictions = np.random.normal(loc=-1, size=len(sentences))
        predicted_sentences = [i for i,j in enumerate(predictions) if j > 0]
        if predicted_sentences:
            predicted_evidence[doc_id] = {"sentences": predicted_sentences}
    claim["predicted_evidence_rand"] = predicted_evidence

# with open("data/random_claims_dev_with_predicted_sentences.jsonl", "w") as outfile:
#     for claim in claims_dev:
#         json.dump(claim, outfile)
#         outfile.write("\n")

In [ ]:
claims_dev[0]["predicted_evidence_rand"]

In [ ]:
##TODO for every claim in the development set, and for every sentence in each retrieved abstract
#predict whether it is evidence or not
for idx, claim in tqdm(enumerate(claims_dev)):
    doc_ids = claim[doc_ids_key]
    predicted_evidence = {}
    for doc_id in doc_ids:
        sentences = corpus[doc_id]["abstract"]
        predictions = []
        for sent in sentences:
            tokens = tokenizer(claim['claim'], sent, return_tensors="pt")
            with torch.no_grad():
                pred = list(model(**tokens)[0].squeeze().numpy())
                predictions.append(pred.index(max(pred)))
        predicted_sentences = [i for i,j in enumerate(predictions) if j > 0]

        if predicted_sentences:
            predicted_evidence[doc_id] = {"sentences": predicted_sentences}
    claims_dev[idx]["predicted_evidence"] = predicted_evidence

# with open("data/claims_dev_with_predicted_sentences.jsonl", "w") as outfile:
#     for claim in claims_dev:
#         json.dump(claim, outfile)
#         outfile.write("\n")

In [ ]:
# some utils to evaluate this using the official metrics for SciFact
from collections import Counter
def safe_divide(num, denom):
    if denom == 0:
        return 0
    else:
        return num / denom

def compute_f1(counts, difficulty=None):
    correct_key = "correct" if difficulty is None else f"correct_{difficulty}"
    precision = safe_divide(counts[correct_key], counts["retrieved"])
    recall = safe_divide(counts[correct_key], counts["relevant"])
    f1 = safe_divide(2 * precision * recall, precision + recall)
    return {"precision": precision, "recall": recall, "f1": f1}

def is_correct(pred_sentence, pred_sentences, gold_sets):
    """
    A predicted sentence is correctly identified if it is part of a gold
    rationale, and all other sentences in the gold rationale are also
    predicted rationale sentences.
    """
    for gold_set in gold_sets:
        gold_sents = gold_set["sentences"]
        if pred_sentence in gold_sents:
            if all([x in pred_sentences for x in gold_sents]):
                return True
            else:
                return False

    return False


def evaluate_sentence_retrieval(dataset, rationale_selection, rand:bool = False):
    counts = Counter()
    for data, retrieval in zip(dataset, rationale_selection):
        assert data['id'] == retrieval['id']

        # Count all the gold evidence sentences.
        for doc_key, gold_rationales in data["evidence"].items():
            for entry in gold_rationales:
                counts["relevant"] += len(entry["sentences"])

        claim_id = retrieval['id']
        for doc_id, pred_sentences in retrieval[f'predicted_evidence' + ('_rand'if rand else '')].items():
            true_evidence_sets = data['evidence'].get(doc_id) or []
            for pred_sentence in pred_sentences:
                counts["retrieved"] += 1
                if is_correct(pred_sentence, pred_sentences, true_evidence_sets):
                    counts["correct"] += 1
    f1 = compute_f1(counts)
    print(f1)
    




In [ ]:
# evaluate
print('random evaluation:')
evaluate_sentence_retrieval(claims_dev, claims_dev, rand = True)
# and we find that our random baseline behaves poorly :(


In [ ]:
##TODO evaluate your predictions
evaluate_sentence_retrieval(claims_dev, claims_dev)

**Part 2 of the Assignment: Claim Verification**

To recap: For every claim, we have retrieved possible evidence sentence. Now, we want to determine whether these sentences support or contradict a claim. Usually, this is handled via textual entailment; if the evidence entails the claim, it is supported (and else, it is contradicted). For this task, you should train your own model, we propose to start from a [distilbert checkpoint](https://huggingface.co/typeform/distilbert-base-uncased-mnli) which has been pre-trained on MNLI. 

Again, we provide a random baseline and evaluate this baseline.

In [ ]:
corpus = {str(doc['doc_id']): doc for doc in jsonlines.open("data/corpus.jsonl")}

In [ ]:
# random baseline

id2label = {0:"SUPPORT", 1: "NOT_ENOUGH_INFO", 2:"CONTRADICT"}
for claim in claims_dev:
    predicted_evidence = claim["predicted_evidence_rand"]
    labels = {}
    for doc_id, sentence_ids in predicted_evidence.items():
        abstract = corpus[doc_id]["abstract"]
        sentences = " ".join(abstract[i] for i in sentence_ids["sentences"])
        label = id2label[np.random.choice([0,1,2])]
        # if we predict neutral, we just ignore these evidence sentences
        labels[doc_id] = {"label_rand": label}
    claim["labels"] = labels
            
        
claims_dev[0]['labels']

In [ ]:
##TODO create an appropriate dataset to fine-tune your model 
# (input to your model should be [CLS] claim [SEP] evidence_sentence [SEP]
# it might be required to sample some evidence sentences which are not annotated and act as "neutral" or 
# "NOT_ENOUGH_INFO" examples
# (hint: labels in mnli are: LABELS = {'CONTRADICT': 0, 'NOT_ENOUGH_INFO': 1, 'SUPPORT': 2}

Create dataset

In [ ]:
label2idx =  {'CONTRADICT': 0, 'NOT_ENOUGH_INFO': 1, 'SUPPORT': 2}


def df_builder(claims):
    for claim in claims:
        for evidence_k, evidence_v in claim['evidence'].items():
            evidence_v:list
            for e in evidence_v:
                for sent_idx in e['sentences']:
                    yield {'claim':claim['claim'], 'evidence_sent': corpus[evidence_k]['abstract'][sent_idx], 'label':label2idx[e['label']]}

In [ ]:
import pandas as pd
df_train = pd.DataFrame(df_builder(claims_train))
df_dev = pd.DataFrame(df_builder(claims_dev))
df_train.head()

Init distilbert

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertConfig,Trainer, TrainingArguments

config = DistilBertConfig.from_pretrained("typeform/distilbert-base-uncased-mnli")
config.num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained("typeform/distilbert-base-uncased-mnli", config=config)
tokenizer = AutoTokenizer.from_pretrained("typeform/distilbert-base-uncased-mnli")

In [ ]:
tokenizer(df_train['claim'][0], df_train['evidence_sent'][0])

In [ ]:
from torch.utils.data import Dataset
class SciFactDataset(Dataset):
    def __init__(self, df, tokenizer):
        super().__init__()
        self.df = df
        self.tokenizer = tokenizer

    def __getitem__(self, index):
        row = self.df.iloc[index]
        return self.tokenizer(row.claim, row.evidence_sent, truncation = True), row.label

    def __len__(self):
        return len(self.df)

In [ ]:
def compute_metric(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}
training_args = TrainingArguments("fine_tuned_distilbert", num_train_epochs=1,load_best_model_at_end=True,evaluation_strategy = "epoch")

In [ ]:
train_dataset = SciFactDataset(df_train, tokenizer)

In [ ]:
train_dataset[0]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=SciFactDataset(df_train, tokenizer),
    eval_dataset=SciFactDataset(df_dev, tokenizer),
    compute_metrics=compute_metric,
    tokenizer=tokenizer,
)

I'm not sure how to create the Dataset class. How to we give the label?

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

Apply pretrained model

In [ ]:
model.eval()
tokens = tokenizer('The earth is round.', 'The earth is flat.', return_tensors="pt")
with torch.no_grad():
    pred = list(model(**tokens)[0].squeeze().numpy())
    print(id2label[pred.index(max(pred))])

In [ ]:
id2label = {0:"SUPPORT", 1: "NOT_ENOUGH_INFO", 2:"CONTRADICT"}
for idx, claim in enumerate(claims_dev):
    predicted_evidence = claim["predicted_evidence"]
    labels = {}
    for doc_id, sentence_ids in predicted_evidence.items():
        abstract = corpus[doc_id]["abstract"]
        # sentences = " ".join(abstract[i] for i in sentence_ids["sentences"])
        predictions = []
        for sent_idx in sentence_ids["sentences"]:
            # print(abstract[sent_idx])
            tokens = tokenizer(claim['claim'], abstract[sent_idx], return_tensors="pt")
            with torch.no_grad():
                pred = list(model(**tokens)[0].squeeze().numpy())
                predictions.append(id2label[pred.index(max(pred))])
        # if we predict neutral, we just ignore these evidence sentences
        labels[doc_id] = {"label": predictions}
    claims_dev[idx]["labels"] = labels


claims_dev[0]['labels']

In [ ]:
# some utils to evalaute this using the official metrics for SciFact
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
def evaluate_labels(dataset, label_prediction):
    LABELS = {'CONTRADICT': 0, 'NOT_ENOUGH_INFO': 1, 'SUPPORT': 2}
    pred_labels = []
    true_labels = []

    for data, prediction in zip(dataset, label_prediction):
        assert data['id'] == prediction['id']

        if not prediction['labels']:
            continue

        claim_id = data['id']
        for doc_id, pred in prediction['labels'].items():
            pred_label = pred['label']
            true_label = {es['label'] for es in data['evidence'].get(doc_id) or []}
            assert len(true_label) <= 1, 'Currently support only one label per doc'
            true_label = next(iter(true_label)) if true_label else 'NOT_ENOUGH_INFO'
            pred_labels.append(LABELS[pred_label])
            true_labels.append(LABELS[true_label])

    print(f'Accuracy           {round(sum([pred_labels[i] == true_labels[i] for i in range(len(pred_labels))]) / len(pred_labels), 4)}')
    print(f'Macro F1:          {f1_score(true_labels, pred_labels, average="macro").round(4)}')
    print(f'Macro F1 w/o NEI:  {f1_score(true_labels, pred_labels, average="macro", labels=[0, 2]).round(4)}')
    print()
    print('                   [C      N      S     ]')
    print(f'F1:                {f1_score(true_labels, pred_labels, average=None).round(4)}')
    print(f'Precision:         {precision_score(true_labels, pred_labels, average=None).round(4)}')
    print(f'Recall:            {recall_score(true_labels, pred_labels, average=None).round(4)}')
    print()
    print('Confusion Matrix:')
    print(confusion_matrix(true_labels, pred_labels))

In [ ]:
##TODO evaluate your own predictions

In [ ]:
evaluate_labels(claims_dev, claims_dev)